In [46]:
import pandas as pd
import numpy as np
from fast_ml.model_development import train_valid_test_split

In [47]:
# kõigi andmete sisselugemine
data1 = pd.read_csv("../../data/avalik_2012_2015.csv", sep='\t')
data2 = pd.read_csv("../../data/avalik_2016_2020.csv", sep='\t')
data3 = pd.read_csv("../../data/avalik_2020_2021.csv", sep='\t')
df = pd.concat([data1, data2, data3])
df.drop_duplicates(subset='JuhtumId', inplace=True)
df.head()

,JuhtumId,ToimKpv,ToimKell,ToimNadalapaev,SyndmusLiik,SyndmusTaiendavStatLiik,Seadus,Paragrahv,ParagrahvTais,Loige,Kahjusumma,KohtLiik,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y,SyyteoLiik
0,ec47da68-1635-18d5-8326-a9a756483dba,2015-12-31,23:55,Neljapäev,VARGUS,MUU_VARGUS,Karistusseadustik,§ 199.,§ 199. Vargus,lg. 1.,0-499,"AVALIK_KOHT,KONNITEE_TEEPEENAR,MUU KOHT",Tartu maakond,Tartu linn,Tartu linn,6474000-6474499,658500-658999,KT
1,ec47a46c-1635-18d5-8326-a9a756483dba,2015-12-31,23:45,Neljapäev,VARGUS,"MOBIILTELEFONIVARGUS,TASKUVARGUS",Karistusseadustik,§ 199.,§ 199. Vargus,lg. 1.,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinn,Kesklinna linnaosa,6589000-6589499,542000-542499,KT
2,ec47a69c-1635-18d5-8326-a9a756483dba,2015-12-31,23:30,Neljapäev,VARGUS,MUU_VARGUS,Karistusseadustik,§ 199.,§ 199. Vargus,lg. 2.,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinn,Kristiine linnaosa,6588000-6588499,540000-540499,KT
3,ec47a5ac-1635-18d5-8326-a9a756483dba,2015-12-31,22:30,Neljapäev,VANDALISM,NaN,Karistusseadustik,§ 218.,§ 218. Varavastane süütegu väheväärtusliku asj...,lg. 1.,500-4999,"AVALIK_KOHT,PARKLA",Harju maakond,Tallinn,Kesklinna linnaosa,6588000-6588499,542000-542499,VT
4,ec494c18-1635-18d5-8326-a9a756483dba,2015-12-31,22:00,Neljapäev,PISIVARGUS,NaN,Karistusseadustik,§ 218.,§ 218. Varavastane süütegu väheväärtusliku asj...,lg. 1.,0-499,"AVALIK_KOHT,OOKLUBI_DISKOTEEK",Harju maakond,Tallinn,Kesklinna linnaosa,6588500-6588999,542000-542499,VT


In [48]:
# ainult oluliste veergude alles jätmine
df.drop(columns=['SyndmusTaiendavStatLiik', 'Seadus', 'Paragrahv', 'ParagrahvTais','Loige', 'SyyteoLiik'], inplace=True)

In [49]:
#kõikides kohtades kus on na aga on oluline et oleks salvestatud siis teeme asju
df = df.dropna()

In [50]:
df.sort_values(by=["ToimKpv", "ToimKell"], inplace=True)
df.head()

,JuhtumId,ToimKpv,ToimKell,ToimNadalapaev,SyndmusLiik,Kahjusumma,KohtLiik,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y
51806,ec2ba64a-1635-18d5-8326-a9a756483dba,2012-01-01,00:00,Pühapäev,VARGUS,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,542000-542499
51807,ec2c5676-1635-18d5-8326-a9a756483dba,2012-01-01,00:00,Pühapäev,VANDALISM,0-499,"AVALIK_KOHT,PARKLA",Pärnu maakond,Sauga vald,Sauga alevik,6476000-6476999,529000-529999
51805,ec2bc86e-1635-18d5-8326-a9a756483dba,2012-01-01,00:15,Pühapäev,VARGUS,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,543500-543999
51804,4294dd58-1636-18d5-8326-a9a756483dba,2012-01-01,00:20,Pühapäev,AVALIKU_KORRA_RIKKUMINE,0-499,MUU KOHT,Harju maakond,Tallinna linn,Lasnamäe linnaosa,6589000-6589499,547000-547499
51803,ec2ba500-1635-18d5-8326-a9a756483dba,2012-01-01,00:30,Pühapäev,VARGUS,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,542000-542499


In [51]:
df.drop(columns="JuhtumId", inplace=True)

In [52]:
df['Toim']= pd.to_datetime(df['ToimKpv'] + ' ' +df['ToimKell'])
df.drop(columns=["ToimKpv", "ToimKell"], inplace=True)

df.dtypes

ToimNadalapaev             object
SyndmusLiik                object
Kahjusumma                 object
KohtLiik                   object
MaakondNimetus             object
ValdLinnNimetus            object
KohtNimetus                object
Lest_X                     object
Lest_Y                     object
Toim               datetime64[ns]
dtype: object

In [53]:
#df.head()
#df[df["SyndmusLiik"].str.contains(",")]
new_df = pd.concat([df.drop('SyndmusLiik', 1), df['SyndmusLiik'].str.get_dummies(sep=",").add_prefix("LIIK_")], 1)
new_df = pd.concat([new_df.drop('KohtLiik', 1), new_df['KohtLiik'].str.get_dummies(sep=",").add_prefix("KOHT_")], 1)
new_df = pd.get_dummies(new_df, columns=["ToimNadalapaev", "Kahjusumma"])
new_df.head()

,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y,Toim,LIIK_AVALIKU_KOOSOLEKU_TEADE,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_ILLEGAALNE_KAUBA_MUUK,LIIK_INTELLEKTUAALSE_OMANDI_OIG_RIKK,...,ToimNadalapaev_Kolmapäev,ToimNadalapaev_Laupäev,ToimNadalapaev_Neljapäev,ToimNadalapaev_Pühapäev,ToimNadalapaev_Reede,ToimNadalapaev_Teisipäev,Kahjusumma_0-499,Kahjusumma_500-4999,Kahjusumma_5000-49999,Kahjusumma_50000-x
51806,Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,542000-542499,2012-01-01 00:00:00,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
51807,Pärnu maakond,Sauga vald,Sauga alevik,6476000-6476999,529000-529999,2012-01-01 00:00:00,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
51805,Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,543500-543999,2012-01-01 00:15:00,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
51804,Harju maakond,Tallinna linn,Lasnamäe linnaosa,6589000-6589499,547000-547499,2012-01-01 00:20:00,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
51803,Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,542000-542499,2012-01-01 00:30:00,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [54]:
#remove useless onehotencoded things
#del new_df_nums

df_numerics_only = new_df.select_dtypes(include=np.number)
#new_df_nums = pd.concat([new_df.drop(columns=new_df.select_dtypes(include=np.number).columns), df_numerics_only[df_numerics_only.columns[df_numerics_only.sum()>100]]])
df1 = new_df.drop(columns=new_df.select_dtypes(include=np.number).columns)
df2 = df_numerics_only[df_numerics_only.columns[df_numerics_only.sum()>100]]
#new_df_nums = new_df.drop(columns=[for col in new_df.select_dtypes(include=np.number).columns if col.sum() < 100])
print(df1.shape)
print(df2.shape)

# SIIN SAAB MUUTA mis on meie tähtsuse piir
to_drop = [col for col in df_numerics_only.columns if df_numerics_only[col].sum() < 100]
print(len(to_drop))
print(to_drop)
new_df.drop(columns=to_drop, inplace=True)

#new_df_nums.shape

(78373, 6)
(78373, 61)
49
['LIIK_AVALIKU_KOOSOLEKU_TEADE', 'LIIK_ILLEGAALNE_KAUBA_MUUK', 'LIIK_INTELLEKTUAALSE_OMANDI_OIG_RIKK', 'LIIK_JUHTUM_LOOMAGA', 'LIIK_JUHTUM_TULIRELVAGA', 'LIIK_JU_LIIK_COVID_19', 'LIIK_JU_LIIK_LIIKUMISPIIRANG', 'LIIK_KADUNUD_ESE/DOKUMENT', 'LIIK_KAUPLEMISE_NOUETE_RIKKUMINE', 'LIIK_LEITUD_LAIP', 'LIIK_LEITUD_LOHKEKEHA(-AINE)', 'LIIK_LIIKLUSONNETUS', 'LIIK_METALLIVARGUS', 'LIIK_METSAVARGUS', 'LIIK_MITTESULARAHALINE_MAKSEVAHEND', 'LIIK_NARKOJUHTUM', 'LIIK_PLAHVATUS', 'LIIK_POMMIAHVARDUS_KAHTLUS', 'LIIK_STREETRACE', 'LIIK_VALIMISED', 'LIIK_VARASTATU_MUUK/HOIDMINE', 'KOHT_HALDUSASUTUS', 'KOHT_INTERNET', 'KOHT_JALAKAIJATUNNEL', 'KOHT_KALMISTU', 'KOHT_KINO', 'KOHT_KIRIK_MUU_KULTUSEHOONE', 'KOHT_KONNITEE_TEEPEENAR', 'KOHT_LENNUJAAM', 'KOHT_LENNUK_OHUSOIDUK', 'KOHT_MAANTEE', 'KOHT_METS', 'KOHT_MUUSEUM', 'KOHT_NAITUSESAAL', 'KOHT_PANK', 'KOHT_PARKIMISMAJA', 'KOHT_PIIRIPUNKT', 'KOHT_RAUDTEE', 'KOHT_RONG', 'KOHT_SAAL_AULA', 'KOHT_SOOJUSTRASS', 'KOHT_STAADION', 'KOHT_SUPELR

In [55]:
koht_df = [col for col in df_numerics_only.columns if col not in to_drop and "KOHT_" in col]
liik_df = [col for col in df_numerics_only.columns if col not in to_drop and "LIIK_" in col]
print(koht_df)
print(liik_df)
print(len(koht_df))
print(len(liik_df))

['KOHT_APTEEK', 'KOHT_AVALIK_KOHT', 'KOHT_BUROO_KONTOR', 'KOHT_BUSSIJAAM_BUSSITERMINAAL', 'KOHT_BUSSIPEATUS', 'KOHT_EHITUS', 'KOHT_HOTELL_MOTELL', 'KOHT_KASIINO', 'KOHT_KAUBAMAJA', 'KOHT_KAUPLUS', 'KOHT_KIOSK', 'KOHT_KULTUURIASUTUS', 'KOHT_LASTEASUTUS_TERRITOORIUM', 'KOHT_MUU KOHT', 'KOHT_MUU RUUM', 'KOHT_OOKLUBI_DISKOTEEK', 'KOHT_OPPEASUTUS_SELLE_TERRITOORIUM', 'KOHT_OUEALATEE', 'KOHT_OUEALA_LAHTINE_HOOV', 'KOHT_PANGAAUTOMAAT', 'KOHT_PARK', 'KOHT_PARKLA', 'KOHT_RAND', 'KOHT_RAUDTEEJAAM', 'KOHT_RAVIASUTUS', 'KOHT_REISILAEV', 'KOHT_RIIDEHOID', 'KOHT_SADAM', 'KOHT_SOOGIKOHT', 'KOHT_SPORDIRAJATIS', 'KOHT_TAKSO', 'KOHT_TANAV_VALJAK', 'KOHT_TANKLA', 'KOHT_TEENINDUSETTEVOTE', 'KOHT_TREPIKODA_KORIDOR', 'KOHT_TURG', 'KOHT_UHISSOIDUK']
['LIIK_AVALIKU_KORRA_RIKKUMINE', 'LIIK_JALGRATTA_MOPEEDI_VARGUS', 'LIIK_KEHALINE_VAARKOHTLEMINE', 'LIIK_KELMUS', 'LIIK_KONFLIKT/OMAVOLI', 'LIIK_KOOLIVAGIVALD', 'LIIK_LIIKLUSRIKKUMINE', 'LIIK_MOOTORSOIDUKI_VARGUS', 'LIIK_MUU', 'LIIK_PISIVARGUS', 'LIIK_ROOVIMINE', 

In [57]:
# eemaldame need read kus kõik need KOHT_ asjade summa on 0
# eemaldame need read kus kõik need LIIK_ asjade summa on 0

# miks? sest need on need read, mis kaotasid oma tähenduse siis kui võtsime ära need onehotencodetud asjad, millel oli alla 100 tegelase
# new_df.shape = (78373, 67)
clean_df = new_df[new_df[koht_df].sum(axis=1) != 0]
clean_df = clean_df[clean_df[liik_df].sum(axis=1) != 0]
# clean_df = 78095 rows × 67 columns



,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y,Toim,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,...,ToimNadalapaev_Esmaspäev,ToimNadalapaev_Kolmapäev,ToimNadalapaev_Laupäev,ToimNadalapaev_Neljapäev,ToimNadalapaev_Pühapäev,ToimNadalapaev_Reede,ToimNadalapaev_Teisipäev,Kahjusumma_0-499,Kahjusumma_500-4999,Kahjusumma_5000-49999
51806,Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,542000-542499,2012-01-01 00:00:00,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
51807,Pärnu maakond,Sauga vald,Sauga alevik,6476000-6476999,529000-529999,2012-01-01 00:00:00,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
51805,Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,543500-543999,2012-01-01 00:15:00,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
51804,Harju maakond,Tallinna linn,Lasnamäe linnaosa,6589000-6589499,547000-547499,2012-01-01 00:20:00,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
51803,Harju maakond,Tallinna linn,Kesklinna linnaosa,6588500-6588999,542000-542499,2012-01-01 00:30:00,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Harju maakond,Tallinn,Põhja-Tallinna linnaosa,6589000-6589499,541500-541999,2021-11-16 15:56:00,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,Harju maakond,Tallinn,Mustamäe linnaosa,6585000-6585499,537000-537499,2021-11-16 16:53:00,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,Harju maakond,Tallinn,Kesklinna linnaosa,6587000-6587499,543000-543499,2021-11-16 18:29:00,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,Harju maakond,Tallinn,Põhja-Tallinna linnaosa,6590000-6590499,541000-541499,2021-11-17 10:35:00,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [58]:
clean_df.to_csv("valmispmst.csv")
#new_df.drop(columns=new_df.select_dtypes(include=np.number).columns).to_csv("stringid.csv")
#df_numerics_only[df_numerics_only.columns[df_numerics_only.sum()>100]].to_csv("yle100.csv")

In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'SalePrice',
                                                                            method='sorted', sort_by_col='saledate',
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)